# Summarize Results

This notebook reads the results from each of the finalized shapefiles and writes the results to a .csv file

In [1]:
import glob
import os
import os.path as op
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import zipfile

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

# Set glacier and ice catchment region numbers
region = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']
catch_region = ['3', '4', '5', '6', '7', '8', '9', '10', '17']

## Glaciers

In [2]:
# Unzip the glacier shapefiles
for x in region:
    my_glacier_zipfile = "data/final-dataset/region-" + x + "-largest-glaciers.zip"
    if x != "7":
        with zipfile.ZipFile(my_glacier_zipfile,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
    if x == "7":
        # Unzip region 7 separately because its file naming convention is different
        jan_mayen_glacier_fp = "data/final-dataset/region-" + x + "-jan-mayen-largest-glaciers.zip"
        with zipfile.ZipFile(jan_mayen_glacier_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
            
        svalbard_glacier_fp = "data/final-dataset/region-" + x + "-svalbard-largest-glaciers.zip"
        with zipfile.ZipFile(svalbard_glacier_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")

In [3]:
# Open finalized glacier shapefiles and concatenate them to a single data frame
for x in region:
    my_glacier_shapefile = "data/final-dataset/unzipped/region-" + x + "-largest-glaciers.shp"
    if x == "1":
        glacier_regions = gpd.read_file(my_glacier_shapefile)
        
    elif x == "7":
        jan_mayen_glacier_shapefile = "data/final-dataset/unzipped/region-" + x + "-jan-mayen-largest-glaciers.shp"
        glacier_regions_part = gpd.read_file(jan_mayen_glacier_shapefile)
        glacier_regions = glacier_regions.append(glacier_regions_part)
        
        svalbard_glacier_shapefile = "data/final-dataset/unzipped/region-" + x + "-svalbard-largest-glaciers.shp"
        glacier_regions_part = gpd.read_file(svalbard_glacier_shapefile)
        glacier_regions = glacier_regions.append(glacier_regions_part)  
        
    else:
        glacier_regions_part = gpd.read_file(my_glacier_shapefile)
        glacier_regions = glacier_regions.append(glacier_regions_part)

In [4]:
# Write glacier dataframe to csv
csv_glacier_fp = "data/final-dataset/compiled-glacier-sizes.csv"
glacier_regions.to_csv(csv_glacier_fp, encoding='utf-8-sig', index=False, columns=['region_no', 'reg_name', 'glac_name', 
                                                                                   'glims_id', 'primeclass', 'area_km2', 
                                                                                   'date'])

## Ice Catchments

In [5]:
# Unzip the ice catchment shapefiles
for x in catch_region:
    my_ic_zipfile = "data/final-dataset/region-" + x + "-largest-complexes.zip"
    if x != "7":
        with zipfile.ZipFile(my_ic_zipfile,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
    if x == "7":
        # Unzip region 7 separately because its file naming convention is different
        jan_mayen_ic_fp = "data/final-dataset/region-" + x + "-jan-mayen-largest-ice-caps.zip"
        with zipfile.ZipFile(jan_mayen_ic_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")
            
        svalbard_ic_fp = "data/final-dataset/region-" + x + "-svalbard-largest-ice-caps.zip"
        with zipfile.ZipFile(svalbard_ic_fp,"r") as zip_ref:
            zip_ref.extractall("data/final-dataset/unzipped")

In [6]:
# Open finalized ice catchment shapefiles and concatenate them to a single data frame
for x in catch_region:
    my_ic_shapefile = "data/final-dataset/unzipped/region-" + x + "-largest-complexes.shp"
    if x == "3":
        ic_regions = gpd.read_file(my_ic_shapefile)
        
    elif x == "7":
        jan_mayen_ic_shapefile = "data/final-dataset/unzipped/region-" + x + "-jan-mayen-largest-ice-caps.shp"
        ic_regions_part = gpd.read_file(jan_mayen_ic_shapefile)
        ic_regions = ic_regions.append(ic_regions_part)
        
        svalbard_ic_shapefile = "data/final-dataset/unzipped/region-" + x + "-svalbard-largest-ice-caps.shp"
        ic_regions_part = gpd.read_file(svalbard_ic_shapefile)
        ic_regions = ic_regions.append(ic_regions_part)  
        
    else:
        ic_regions_part = gpd.read_file(my_ic_shapefile)
        ic_regions = ic_regions.append(ic_regions_part)

In [7]:
# Write glacier dataframe to csv
csv_fp = "data/final-dataset/compiled-catchment-sizes.csv"
ic_regions.to_csv(csv_fp, encoding='utf-8-sig', index=False, columns=['region_no', 'reg_name', 'ic_name', 
                                                'primeclass', 'area_km2', 'min_date', 'max_date'])

## Clean up

In [8]:
# Clean up unzipped files to save disk space
filelist = glob.glob("data/final-dataset/unzipped/*")
for f in filelist:
    os.remove(f)